# Fantasy Football - Optimal Drafting Guide and Predictions

### Goals:  
1. Create a PPR drafting strategy guide.
    - Round by round analysis
    - General drafting principles
      
2. Create 2023 player ppr projections and tier list

In [1]:
# Imports
import pandas as pd
import numpy as np
import time

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.model_selection import TimeSeriesSplit

In [2]:
df = pd.read_csv('season.csv',index_col=0)
df = df.sort_values('year').reset_index(drop=True)